# キャッシュ

- 著者: [Joseph](https://github.com/XaviereKU)
- 査読: [Teddy Lee](https://github.com/teddylee777)、[BAEM1N](https://github.com/BAEM1N)
- 校正: [Two-Jay](https://github.com/Two-Jay)
- これは[LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/04-Model/03-Cache.ipy...)

## 概要

```LangChain```はLLM向けのオプションのキャッシュレイヤーを提供します。

これは次の2つの理由で有用です:
- 同じ補完（completion）を複数回要求する場合、LLMプロバイダーへのAPIコール回数を削減でき、結果としてコストを節約できます。
- LLMプロバイダーへのAPIコール回数を減らすことで、アプリケーションの実行時間を改善できます。

本チュートリアルでは、OpenAIの `gpt-4o-mini` を使用し、`InMemoryCache` と `SQLiteCache` の2種類のキャッシュを利用します。
各セクションの最後で、キャッシュ導入前後の実行時間（Wall time）を比較します。

### 目次

- [概要](#overview)
- [環境設定](#environment-setup)
- [InMemoryCache](#inmemorycache)
- [SQLiteCache](#sqlitecache)

### 参考
- [SQLiteCache](https://python.langchain.com/api_reference/community/cache/langchain_community.cache.SQLiteCache.html#langchain_community.cache.SQLiteCache)
- [InMemoryCache](https://python.langchain.com/api_reference/core/caches/langchain_core.caches.InMemoryCache.html)
----

## 環境設定

環境をセットアップします。詳細は [Environment Setup](https://wikidocs.net/257836) を参照してください。

**[注意]**
- `langchain-opentutorial` はチュートリアル向けの便利な環境設定やユーティリティを提供するパッケージです。
- `langchain-opentutorial` の詳細は [langchain-opentutorial](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) を参照してください。

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
        # "vllm", # this is for optional section
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "You OpenAI API KEY",
        "LANGCHAIN_API_KEY": "LangChain API KEY",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Caching",
    }
)

Environment variables have been set successfully.


In [4]:
# Alternatively, one can set environmental variables with load_dotenv
from dotenv import load_dotenv


load_dotenv(override=True)

False

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Create model
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Generate prompt
prompt = PromptTemplate.from_template(
    "Sumarize about the {country} in about 200 characters"
)

# Create chain
chain = prompt | llm

In [6]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea, located on the Korean Peninsula, is known for its rich culture, advanced technology, and vibrant economy. It features bustling cities like Seoul, renowned cuisine, and historic lan[...]
CPU times: total: 93.8 ms
Wall time: 1.54 s


## `InMemoryCache`
まず、`InMemoryCache` を使って同じ質問の答えをキャッシュします。

In [18]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

# Set InMemoryCache
set_llm_cache(InMemoryCache())

In [8]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country known for its fast-paced lifestyle, vibrant culture, and delicious cuisine. It is a leader in industries such as electronics, automotive, and e[...]
CPU times: total: 0 ns
Wall time: 996 ms


同じ質問でチェーンを呼び出します。

In [10]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country known for its fast-paced lifestyle, vibrant culture, and delicious cuisine. It is a leader in industries such as electronics, automotive, and e[...]
CPU times: total: 0 ns
Wall time: 3 ms


注意: 再度 `InMemoryCache` を設定するとキャッシュは失われ、実行時間（Wall time）が増加します。

In [11]:
set_llm_cache(InMemoryCache())

In [12]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a tech-savvy, modern country known for its vibrant culture, delicious cuisine, and booming economy. It is a highly developed nation with advanced infrastructure, high standards o[...]
CPU times: total: 0 ns
Wall time: 972 ms


## `SQLiteCache`
次に、`SQLiteCache` を使って同じ質問の答えをキャッシュします。

In [13]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [14]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia, known for its booming economy, vibrant pop culture, and rich history. It is home to K-pop, Samsung, and delicious cuisine like ki[...]
CPU times: total: 31.2 ms
Wall time: 953 ms


同じ質問でチェーンを呼び出します。

In [15]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia, known for its booming economy, vibrant pop culture, and rich history. It is home to K-pop, Samsung, and delicious cuisine like ki[...]
CPU times: total: 375 ms
Wall time: 375 ms


SQLiteCache を使用すると、再度キャッシュを設定しても保存されたキャッシュは削除されない点に注意してください。

In [16]:
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [17]:
%%time
# Invoke chain
response = chain.invoke({"country": "South Korea"})
print(response.content)

South Korea is a technologically advanced country in East Asia, known for its booming economy, vibrant pop culture, and rich history. It is home to K-pop, Samsung, and delicious cuisine like ki[...]
CPU times: total: 0 ns
Wall time: 4.01 ms
